In [1]:
import pandas as pd

In [2]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [3]:
data_train

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


In [4]:
data_train.iloc[0]


,0
Class Index,3
Title,Wall St. Bears Claw Back Into the Black (Reuters)
Description,"Reuters - Short-sellers, Wall Street's dwindli..."


In [5]:
data_train.isna().sum()

,0
Class Index,0
Title,0
Description,0


In [6]:
data_train.duplicated().sum()

np.int64(0)

In [7]:
data_train.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [8]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder


# دمج العنوان والوصف في نص واحد
texts_train = (data_train['Description'].fillna('') + ' ' + data_train['Title'].fillna('')).tolist()
texts_test = (data_test['Description'].fillna('') + ' ' + data_test['Title'].fillna('')).tolist()



In [9]:
max_words = 20000  # أقصى عدد كلمات في القاموس
max_len = 100      # طول الجملة الأقصى (padding)

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts_train)

X_train_seq = tokenizer.texts_to_sequences(texts_train)
X_test_seq = tokenizer.texts_to_sequences(texts_test)

# جعل كل الجمل بنفس الطول
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')


In [10]:
# استبدل TargetColumn باسم العمود الذي يحتوي التصنيف
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(data_train['Class Index'])

# تحويل إلى One-hot
from tensorflow.keras.utils import to_categorical
y_train_cat = to_categorical(y_train)

In [11]:

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(y_train_cat.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
from sklearn.model_selection import train_test_split

# Split the training data into training and validation sets
X_train_split, X_val_pad, y_train_split, y_val = train_test_split(
    X_train_pad, y_train_cat, test_size=0.2, random_state=42
)


history = model.fit(
    X_train_split, y_train_split,
    epochs=7,
    batch_size=64,
    validation_data=(X_val_pad, y_val)
)

Epoch 1/7
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 395s 261ms/step - accuracy: 0.2502 - loss: 1.3875 - val_accuracy: 0.2505 - val_loss: 1.3872
Epoch 2/7
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 441s 261ms/step - accuracy: 0.2514 - loss: 1.3871 - val_accuracy: 0.2507 - val_loss: 1.3847
Epoch 3/7
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 398s 265ms/step - accuracy: 0.2517 - loss: 1.3846 - val_accuracy: 0.2485 - val_loss: 1.3845
Epoch 4/7
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 397s 265ms/step - accuracy: 0.3597 - loss: 1.2130 - val_accuracy: 0.6662 - val_loss: 0.6753
Epoch 5/7
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 400s 267ms/step - accuracy: 0.7076 - loss: 0.5919 - val_accuracy: 0.8818 - val_loss: 0.3855
Epoch 6/7
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 440s 265ms/step - accuracy: 0.9119 - loss: 0.3055 - val_accuracy: 0.9123 - val_loss: 0.2841
Epoch 7/7
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 429s 257ms/step - accuracy: 0.9400 - loss: 0.2141 - val_accuracy: 0.9144 - val_loss: 0.2792


In [13]:
y_pred = model.predict(X_test_pad)


238/238 ━━━━━━━━━━━━━━━━━━━━ 13s 53ms/step


In [14]:
y_pred_classes = np.argmax(y_pred, axis=1)

# إذا كانت test labels متوفرة
if 'Class Index' in data_test.columns:
    y_test = label_encoder.transform(data_test['Class Index'])

    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

    print("Accuracy:", accuracy_score(y_test, y_pred_classes))
    print("Classification Report:\n", classification_report(y_test, y_pred_classes))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_classes))

Accuracy: 0.9118421052631579
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.89      0.91      1900
           1       0.96      0.97      0.96      1900
           2       0.89      0.87      0.88      1900
           3       0.87      0.91      0.89      1900

    accuracy                           0.91      7600
   macro avg       0.91      0.91      0.91      7600
weighted avg       0.91      0.91      0.91      7600

Confusion Matrix:
 [[1690   65   80   65]
 [  24 1852   16    8]
 [  59   10 1654  177]
 [  52   12  102 1734]]


In [15]:
# إضافة هذا الكود في نهاية الـ notebook الأصلي لحفظ المكونات اللازمة
import pickle
import joblib

# حفظ النموذج
model.save('models/lstm_classifier.h5')

# حفظ الـ tokenizer
with open('models/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# حفظ الـ label encoder
with open('models/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# حفظ المعاملات المهمة
config = {
    'max_words': max_words,
    'max_len': max_len,
    'class_names': {0: 'World', 1: 'Sports', 2: 'Business', 3: 'Sci/Tech'}  # حسب البيانات
}

with open('models/config.pkl', 'wb') as f:
    pickle.dump(config, f)

print("تم حفظ جميع المكونات بنجاح!")

تم حفظ جميع المكونات بنجاح!
